In [1]:
import pandas as pd
import numpy as np
import warnings
import pickle
from math import sqrt
warnings.filterwarnings('ignore')
from azureml.core.run import Run
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.model import Model
from azureml.core.authentication import ServicePrincipalAuthentication

In [2]:
# Connect to Workspace
ws = Workspace.from_config()
print(ws)

Workspace.create(name='MLOps_WS', subscription_id='6faa9ede-4786-48dc-9c1e-0262e2844ebf', resource_group='Learn_MLOps')


## Download scaler and model from workspace

In [3]:
scaler = Model(ws,'scaler').download(exist_ok=True)

In [4]:
svc_model = Model(ws,'support-vector-classifier').download(exist_ok=True)

## Load files

In [5]:
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [6]:
# Compute the prediction with ONNX Runtime
import onnxruntime as rt
import numpy
sess = rt.InferenceSession("svc.onnx")

In [7]:
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

## inference on test data

In [8]:
test_data = np.array([34.927778, 0.24, 7.3899, 83, 16.1000, 1])

In [9]:
# Scale data
test_data = scaler.fit_transform(test_data.reshape(1, 6))

In [10]:
pred_onx = sess.run([label_name], {input_name: test_data.astype(numpy.float32)})[0]

In [11]:
pred_onx[0]

1